In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
    }

def get_feed_article_titles_df(feedname,url):
    """
    Get article titles and create a DataFrame.
    
    Args:
        feedname (str): Name of the feed.
        url (str): URL of the XML feed.
    
    Returns:
        pandas.DataFrame: DataFrame containing article titles and feed name.
    """
    try:
        result = requests.get(url, headers=headers)

        soup = BeautifulSoup(result.text, "xml")        
        article_urls = [i.text for i in soup.findAll('link')]

        #The verge has the links in the id tag, if the list is empty with the link tag, try the id tag
        if len([item for item in article_urls if bool(item)])  == 0: 
            article_urls = [i.text for i in soup.findAll('id')]      
        
       
        #Parse it as html to get the links correctly, other wise In some websites, <media:title> is also returned as a link
        soup = BeautifulSoup(result.text, "html.parser")
        article_titles = [i.text for i in soup.findAll('title')]      
        
        df = pd.DataFrame({'Article_title': article_titles, 'Article_URL': article_urls[-len(article_titles):], 'Feedname': feedname})
        
        #Remove homepage from url list and empty url rows
        homepage = url.split('.com')[0] + '.com/'
        df = df[(df['Article_URL'] != homepage) & (df['Article_URL'] != '') ]        
        
        # Drop duplicate URLs
        df = df.drop_duplicates(subset=['Article_URL'], keep='first')

        df['Fetch_Date'] = str(datetime.datetime.now())

        return df

    except Exception as e:
        print("Error getting feed: ", e)
        return pd.DataFrame()

def get_article_text(url):
    try:
        result = requests.get(url[0][0], headers=headers)
        soup = BeautifulSoup(result.text, "html.parser")   
        return (soup.text)     
    except Exception as e:
            print(e)
            return None
    

In [2]:
import sqlite3

db_name = 'RssFeeds.db'

def get_connection():
    """
    Establish a connection to a SQLite database.
    
    Args:
        db_name (str): Name of the SQLite database file.
    
    Returns:
        sqlite3.Connection: Connection object to the SQLite database.
    """
    try:
        con = sqlite3.connect(db_name)
        return con
    except sqlite3.Error as e:
        print("Error connecting to database: ", e)
        return None

def create_db():
    """
    Create a new SQLite database and execute the given query to create tables.
    
    Args:
        db_name (str): Name of the SQLite database file.
        query (str): SQL query to create tables in the database.
    
    Returns:
        sqlite3.Connection: Connection object to the SQLite database.
    """
    
    query = "CREATE TABLE IF NOT EXISTS FEEDS( Feedname, Article_title UNIQUE, Article_URL, Duplicate, Fetch_Date, Summary)"    

    con = get_connection()
    
    if con is None:
        return None
    
    try:
        cur = con.cursor()
        cur.execute(query)
        con.commit()
        #return con
    except sqlite3.Error as e:
        print("Error creating database: ", e)
        con.close()
        #return None
    
    print("DB created successfully")

def insert_to_db(data, query):
    """
    Insert data into SQLite database using executemany.
    
    Args:
        con (sqlite3.Connection): Connection object to the SQLite database.
        data (list of tuples): Data to be inserted into the database.
        query (str): SQL query for insertion.
    
    Returns:
        None
    """
    con = get_connection()

    if not data:
        print("No data to insert.")
        return
    
    try:
        cur = con.cursor()
        cur.executemany(query, data)
        con.commit()
    except sqlite3.Error as e:
        print("Error inserting data into database: ", e)
        con.rollback()

def insert_to_FEEDS(data):
    con = get_connection()

    if len(data) == 0:
        print("No data to insert.")
        return
    
    try:
        cur = con.cursor()

        query = "INSERT OR REPLACE INTO FEEDS(Article_title,Article_URL,Feedname,Fetch_Date) VALUES (?, ?, ?, ?)"

        cur.executemany(query, data)
        con.commit()
        con.close()
    except sqlite3.Error as e:
        print("Error inserting data into database: ", e)
        con.rollback()
        con.close()

def delete_from_db(tablename):
    """
    Delete data from SQLite database.
    
    Args:
        con (sqlite3.Connection): Connection object to the SQLite database.
        query (str): SQL query for deletion.
    
    Returns:
        None
    """
    con = get_connection()
    query = "DROP TABLE IF EXISTS " + tablename
    try:
        cur = con.cursor()
        cur.execute(query)
        con.commit()
    except sqlite3.Error as e:
        print("Error deleting data from database: ", e)
        con.rollback()


def query_db(query):
    """
    Execute a SQL query and fetch results from SQLite database.
    
    Args:
        con (sqlite3.Connection): Connection object to the SQLite database.
        query (str): SQL query to be executed.
    
    Returns:
        list of tuples: Result set fetched from the database.
    """
    con = get_connection()

    try:
        cur = con.cursor()
        cur.execute(query)
        return cur.fetchall()
    except sqlite3.Error as e:
        print("Error executing query: ", e)
        return []
    

In [3]:
feedlist={'Engadget':'https://www.engadget.com/rss.xml', 
          'The Verge':'https://www.theverge.com/rss/index.xml',
          'Techcrunch':'https://techcrunch.com/feed/',
          'Ars Technica':'https://feeds.arstechnica.com/arstechnica/index',
          'Jalopnik':'https://jalopnik.com/rss'}  

def refresh_feeds():
    """
    Refresh feeds by retrieving and inserting data for each feed in the feedlist.

    This function iterates over each feed in the feedlist, retrieves article data,
    and inserts it into the corresponding database table.

    Note:
    - The feedlist dictionary should contain feed names as keys and their URLs as values.
    - The `get_feed_articles_df` function is expected to return a DataFrame with article titles
      and URLs for a given feed.
    - The `insert_to_FEEDS` function is expected to insert the DataFrame values into the 
      corresponding database table.

    Example usage:
    refresh_feeds()

    """
    for feed in feedlist:
        print('Getting and inserting data for', feed)
        df = get_feed_article_titles_df(feed, feedlist[feed])        
        insert_to_FEEDS(df.values)
    
    df = pd.read_sql("Select * from FEEDS", get_connection())
    df.to_csv('feeds.csv', index=False)


In [4]:
create_db()

DB created successfully


In [5]:
refresh_feeds()

Getting and inserting data for Engadget


c:\Users\lokes\AppData\Local\Programs\Python\Python310\lib\html\parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Getting and inserting data for The Verge
Getting and inserting data for Techcrunch


c:\Users\lokes\AppData\Local\Programs\Python\Python310\lib\html\parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
c:\Users\lokes\AppData\Local\Programs\Python\Python310\lib\html\parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` i

Getting and inserting data for Ars Technica
Getting and inserting data for Jalopnik


c:\Users\lokes\AppData\Local\Programs\Python\Python310\lib\html\parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
c:\Users\lokes\AppData\Local\Programs\Python\Python310\lib\html\parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` i

In [6]:
url = query_db("Select Article_URL from FEEDS where Feedname == 'Engadget' limit 1")
get_article_text(url)

"The best smartphones to buy in 2024 EngadgetLoginReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest Mint AlternativesBest VPNBest Bluetooth TrackersView all Buying GuidesGamingPlayStationNintendoXboxPC GamingAll Gaming NewsGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowScienceSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsFacebookTwitterYouTubeSectionsReviewsBuying GuidesGamingGearEntertainmentTomorrowDealsNewsVideoPodcastsLoginAdvertisementiPad Pro M4 hands-onNintendo hints at 'Switch 2' unveilingGoogle Pixel 8a hands-onHelldivers 2 PSN account reversalBest college graduation giftsRead full articleWhy you can trust usEngadget has been testing and reviewing consumer tech since 2004. Our stories may include affiliate links; if you buy something through a link, we may earn a commission. Read more about how we evaluate prod

In [9]:
df = pd.read_sql("Select * from FEEDS", get_connection())
df.head()
df.shape

(153, 6)